Copyright 2018, Skoltech DeepQauntum Labs, All rights reserved.

# 3SAT Problem and Thermal states

Propositional satisfiability (SAT) is the problem of determining the satisfiability of sentences in propositional logic. If $k$ is the number of literals in each clause the problem is called $K$-SAT. Determining the satisfiability of a formula in conjunctive normal form where each clause is limited to at most $k$ literals is NP-complete. K-satisfiability problems can be reduced---generally called Karp reduction---to 3-satisfiability (3-SAT) when formulas in CNF are considered with each clause containing up to 3 literals.



### CNF representation of 3-SAT
Conjunctive normal form (CNF) is a conjunction of one or more clauses.<br/>
Example 3-SAT problem in CNF form is
$$
(x_{i} \lor \overline{x}_{j} \lor x_{k})\land  (\overline{x}_{g} \lor x_{r} \lor \overline{x}_{s}) \land(\cdots)\land \cdots.
$$

How we can encode cnf form to array?
We can do the following procedure. Every clause can be rewritten as an array $[(\pm)x_i, (\pm)x_j, (\pm)x_k]$, where $-$ for negation. 
<br/>
For example clause $(x_{i} \lor \overline{x}_{j} \lor x_{k})$ can be rewritten as $[i,-j,k]$. Full set of clauses can be considered as the set of arrays.
<br/>
The function below randomly generates 3-SAT with m clauses and n variables

For example the following $3$-SAT problem
\begin{equation}\label{sat_ex}
(x_{1} \lor \overline{x}_{2} \lor x_{4})\land  ({x}_{1} \lor x_{3} \lor \overline{x}_{5})
\end{equation}
is satisfiable for $x_1=1,\,x_2=1,\,x_3=0,\,x_4=0,\,x_5=0$.


In [17]:
import numpy as np
from random import randint
import random
import scipy.linalg as la
import scipy.sparse as sparse
import scipy.sparse.linalg as lasp
import scipy


def cnf_random_generator(n_variables,m_clauses):
    """
    Returns a random 3-SAT problem in CNF form 
    (see https://en.wikipedia.org/wiki/Boolean_satisfiability_problem#3-satisfiability)

    Args:
    ----------
        n_variables (int): The number of boolean variables of 3-SAT problem.
        m_clauses (int): The number of clauses of CNF form of 3-SAT problem.

    Returns:
    ----------
        numpy array: random 3-SAT problem in CNF form
    """    
    cnf=[]
    for i in range(m_clauses):
        a=np.array(sorted(random.sample(range(1,n_variables+1),3)))
        b=np.array([(-1)**randint(0,1),(-1)**randint(0,1),(-1)**randint(0,1)])
        cnf.append(a*b)
    
    return cnf

Generate Random 3-SAT problem with 20 variables and 100 clauses

### How to transform 3-SAT problem from CNF form to Hamiltonian?
Our goal is to transform the 3-SAT problem to Hamiltonian and then cool the system.
<br/>
To do this we need to transform every clause to operator form to make penalty function.
$$(x_{i} \lor \overline{x}_{j} \lor x_{k}) \mapsto 
\vert 0\rangle\!\langle 0\vert_i \otimes \vert 1\rangle\!\langle 1\vert_j \otimes \vert 0\rangle\!\langle 0\vert_k
$$
and then sum all these operators. 
<br/>
$$
P_0=\vert 0\rangle\!\langle 0\vert,\\
P_1=\vert 1\rangle\!\langle 1\vert.
$$
Since the 3-SAT problem consists of n variables then the clause operator has the following form
$$
(x_{i} \lor \overline{x}_{j} \lor x_{k}) \mapsto I_1\otimes\ldots\otimes I_{i-1} \otimes \vert 0\rangle\!\langle 0\vert_i \otimes I_{i+1}\cdots \otimes I_{j-1}\otimes \vert 1\rangle\!\langle 1\vert_j \otimes I_{j+1}\cdots \otimes I_{k-1} \otimes \vert 0\rangle\!\langle 0\vert_k \otimes I_{k+1}\cdots \otimes I_{n}
$$

In [18]:
# Define projectors as diagonal matrix - array
P_0=np.array([[1,0],[0,0]])
P_1=np.array([[0,0],[0,1]])
I=np.array([[1,0],[0,1]])

In [19]:
def clause_to_hamiltonian(clause,n_variables):
    """
    Returns a Hamiltonian of clause of 3-SAT problem in CNF form 
    Args:
    ----------
        clause (1D numpy array or list): The clause.
        n_variables (int): The number of variables of the 3-SAT problem.

    Returns:
    ----------
        numpy array: Hamiltonian of clause of 3-SAT problem
    """
    if np.sign(clause[0])==1:
        H=P_0
    else:
        H=P_1
    for i in range(1,np.abs(clause[0])):
        H=np.kron(I,H)
    
    if np.sign(clause[1])==1:
        H_1=P_0
    else:
        H_1=P_1        
    for i in range(np.abs(clause[0])+1,np.abs(clause[1])):
        H=np.kron(H,I)
    H=np.kron(H,H_1)
    
    if np.sign(clause[2])==1:
        H_2=P_0
    else:
        H_2=P_1        
    for i in range(np.abs(clause[1])+1,np.abs(clause[2])):
        H=np.kron(H,I)
    H=np.kron(H,H_2)
    
    for i in range(np.abs(clause[2])+1, n_variables+1):
        H=np.kron(H,I)
    
    return H

### Problem: 
Write a function to transform 3-SAT problem to Hamiltonian

# Thermal states

Thermal state is defined by 
$$
\frac{e^{-\beta H}}{tr e^{-\beta H}},
$$
where $H$ is a Hamiltonian, $\beta$ is a control parameter, $tr$ is a trace.
<br/>
The overlap between the ground space is:
$$
\sum_{i=1}^{N}\langle i\vert \frac{e^{-\beta H}}{tr e^{-\beta H}}\vert i \rangle,
$$
where $N$ is the degeneracy of ground level and $i$ are ground states

## Problem
Number of variables equals to 8. For every number of clauses from 1 to 100 generate randomly 50 different 3-SAT problems and calculate overlap between thermal state of 3-SAT and the ground state space
<br/>
Plot the graph $\frac{\text{number of clauses}}{\text{number of variables}}$ vs. Average overlap for $\beta \in [3,4,5]$